### Prueba de modelos iniciales

Vectorizo el texto usando TF-IDF solo en datos de entrenamiento para evitar data leakeage

In [189]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime


In [190]:
# Inicializar el vectorizador
vectorizer = TfidfVectorizer(max_features=10_000)  # Usa solo las 10,000 palabras más frecuentes


# Aplicar TF-IDF a la columna 'text'
tfidf_matrix = vectorizer.fit_transform(train_df['text_limpio']).astype('float32')


# Convertir a DataFrame para mejor visualización (opcional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print(tfidf_df.head())  # Ver los primeros resultados


    aa  aaa  abdomen  ability  able  aboutbr  abr  abrasive  absolute  \
0  0.0  0.0      0.0      0.0   0.0      0.0  0.0       0.0       0.0   
1  0.0  0.0      0.0      0.0   0.0      0.0  0.0       0.0       0.0   
2  0.0  0.0      0.0      0.0   0.0      0.0  0.0       0.0       0.0   
3  0.0  0.0      0.0      0.0   0.0      0.0  0.0       0.0       0.0   
4  0.0  0.0      0.0      0.0   0.0      0.0  0.0       0.0       0.0   

   absolutely  ...  zippy  zit  zizyphus  zone  zones  zonesbr  zoom  zoomed  \
0    0.000000  ...    0.0  0.0       0.0   0.0    0.0      0.0   0.0     0.0   
1    0.079113  ...    0.0  0.0       0.0   0.0    0.0      0.0   0.0     0.0   
2    0.000000  ...    0.0  0.0       0.0   0.0    0.0      0.0   0.0     0.0   
3    0.000000  ...    0.0  0.0       0.0   0.0    0.0      0.0   0.0     0.0   
4    0.000000  ...    0.0  0.0       0.0   0.0    0.0      0.0   0.0     0.0   

   zoya  zum  
0   0.0  0.0  
1   0.0  0.0  
2   0.0  0.0  
3   0.0  0.0  
4   0

In [191]:
vectorizer = TfidfVectorizer()

# Aplicar fit_transform SOLO en el conjunto de entrenamiento
X_train_tfidf = vectorizer.fit_transform(train_df['text_limpio'])

# Transformar el conjunto de prueba sin volver a ajustar
X_test_tfidf = vectorizer.transform(test_df['text_limpio'])

# Obtener etiquetas
y_train = train_df['rating']
y_test = test_df['rating']


Codigo para guardar los modelos en formato Pickle en la carpeta Models

In [193]:
import pickle
import os

# Definir la ruta de guardado
model_path = r'C:\Users\Vicky\Documents\ML_Amazon_Reviews_Analysis\models'


In [194]:
#### B. Balanceo con tecnica de SMOTE aplicado al conjunto de entrenamiento vectorizado
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

# Verificar el balanceo
print("Distribución original de y_train:", Counter(y_train))
print("Distribución balanceada de y_train_smote:", Counter(y_train_smote))

# Ahora puedes usar:
# X_train_smote, y_train_smote para entrenamiento
# X_test_tfidf, y_test para evaluación

Distribución original de y_train: Counter({5: 3909, 4: 1166, 3: 658, 1: 533, 2: 378})
Distribución balanceada de y_train_smote: Counter({5: 3909, 3: 3909, 1: 3909, 2: 3909, 4: 3909})


### Entrenamiento y evaluacion de modelos

#### Naive Bayesiano (con SMOTE)

In [196]:
nb_model = MultinomialNB()
nb_model.fit(X_train_smote, y_train_smote)
y_pred_nb = nb_model.predict(X_test_tfidf)

print("Naïve Bayes Performance:")
print(classification_report(y_test, y_pred_nb))


Naïve Bayes Performance:
              precision    recall  f1-score   support

           1       0.60      0.47      0.53       135
           2       0.23      0.40      0.29        93
           3       0.25      0.42      0.32       165
           4       0.34      0.44      0.38       311
           5       0.89      0.66      0.76       958

    accuracy                           0.57      1662
   macro avg       0.46      0.48      0.45      1662
weighted avg       0.66      0.57      0.60      1662



Guardar el modelo en formato pickle

In [198]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y_train)  # Asegúrate de haberlo ajustado con los datos correctos


LabelEncoder()

In [199]:
# preparar todos los componentes para guardar
model_components = {
    'model': nb_model,
    'vectorizer': vectorizer,  # El vectorizador TF-IDF ya fit
    'label_encoder': le,       # El LabelEncoder ya fit
    'metadata': {
        'accuracy': accuracy_score(y_test, y_pred_nb),
        'classification_report': classification_report(y_test, y_pred_nb, output_dict=True),
        'saved_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
}

# 4. Guardar todo en un archivo .pkl
filename = 'naive_bayes_model.pkl'
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"Modelo Naïve Bayes guardado exitosamente en: {full_path}")
print(f"Accuracy del modelo guardado: {accuracy_score(y_test, y_pred_nb):.4f}")

Modelo Naïve Bayes guardado exitosamente en: C:\Users\Vicky\Documents\ML_Amazon_Reviews_Analysis\models\naive_bayes_model.pkl
Accuracy del modelo guardado: 0.5656


#### Regresion logistica

In [201]:
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)

print("Regresión Logística Performance:")
print(accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Regresión Logística Performance:
0.6582430806257521
              precision    recall  f1-score   support

           1       0.69      0.44      0.54       135
           2       0.54      0.08      0.13        93
           3       0.47      0.24      0.32       165
           4       0.45      0.21      0.29       311
           5       0.69      0.96      0.80       958

    accuracy                           0.66      1662
   macro avg       0.57      0.39      0.42      1662
weighted avg       0.62      0.66      0.60      1662



In [202]:
model_components = {
    'model': lr_model,
    'vectorizer': vectorizer,  # El vectorizador TF-IDF ya fit
    'label_encoder': le,       # El LabelEncoder ya fit
    'metadata': {
        'model_type': 'Logistic Regression',
        'accuracy': accuracy_score(y_test, y_pred_lr),
        'classification_report': classification_report(y_test, y_pred_lr, output_dict=True),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d"),
        'features': f"{X_train_tfidf.shape[1]} features TF-IDF",
        'classes': list(le.classes_)
    }
}

# 3. Guardar todo en un archivo .pkl
filename = 'logistic_regression_model.pkl'
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"✅ Modelo de Regresión Logística guardado exitosamente en:\n{full_path}")
print(f"\n📊 Métricas del modelo guardado:")
print(f"- Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"- Classes: {list(le.classes_)}")

✅ Modelo de Regresión Logística guardado exitosamente en:
C:\Users\Vicky\Documents\ML_Amazon_Reviews_Analysis\models\logistic_regression_model.pkl

📊 Métricas del modelo guardado:
- Accuracy: 0.6582
- Classes: [1, 2, 3, 4, 5]


#### SVM (Support Vector Machine)

In [204]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)

print("SVM Performance:")
print(accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM Performance:
0.6648616125150422
              precision    recall  f1-score   support

           1       0.66      0.57      0.61       135
           2       0.37      0.11      0.17        93
           3       0.39      0.25      0.31       165
           4       0.49      0.21      0.30       311
           5       0.71      0.95      0.82       958

    accuracy                           0.66      1662
   macro avg       0.52      0.42      0.44      1662
weighted avg       0.62      0.66      0.61      1662



In [205]:
# 2. Preparar todos los componentes para guardar
model_components = {
    'model': svm_model,
    'vectorizer': vectorizer,  # El vectorizador TF-IDF ya fit
    'label_encoder': le,       # El LabelEncoder ya fit
    'metadata': {
        'model_type': 'Support Vector Machine (Linear Kernel)',
        'accuracy': accuracy_score(y_test, y_pred_svm),
        'classification_report': classification_report(y_test, y_pred_svm, output_dict=True),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d"),
        'features': f"{X_train_tfidf.shape[1]} features TF-IDF",
        'classes': list(le.classes_),
        'model_parameters': svm_model.get_params()
    }
}

# 3. Guardar todo en un archivo .pkl
filename = 'svm_linear_model.pkl'
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"✅ Modelo SVM (Linear Kernel) guardado exitosamente en:\n{full_path}")
print(f"\n📊 Métricas del modelo guardado:")
print(f"- Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"- Classes: {list(le.classes_)}")
print(f"- Parámetros: {svm_model.get_params()}")

✅ Modelo SVM (Linear Kernel) guardado exitosamente en:
C:\Users\Vicky\Documents\ML_Amazon_Reviews_Analysis\models\svm_linear_model.pkl

📊 Métricas del modelo guardado:
- Accuracy: 0.6649
- Classes: [1, 2, 3, 4, 5]
- Parámetros: {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


# Análisis de Métricas de Modelos de Clasificación

A continuación, se presenta un análisis simplificado de las métricas de rendimiento para tres modelos de clasificación: Naïve Bayes, Regresión Logística y SVM.

## Naïve Bayes Performance:

### Naïve Bayes

* **Precisión General:** 57% (aceptable, un poco más de la mitad de las clasificaciones son correctas).
* **Clase 5:** Buen rendimiento (alta precisión y exhaustividad).
* **Clases 1, 2, 3, 4:** Rendimiento bajo (dificultad para identificar correctamente estas clases).
* **Promedio Macro:** Bajo (problemas generales con el rendimiento en todas las clases, especialmente las minoritarias).

### Regresión Logística


**Análisis:**

* **Precisión General:** 66% (mejor que Naïve Bayes).
* **Clase 5:** Muy buen rendimiento (alta exhaustividad).
* **Clases 2, 3, 4:** Baja exhaustividad (el modelo pasa por alto muchos casos de estas clases).
* **Promedio Macro:** Mejor que Naïve Bayes, pero aún bajo (dificultades con las clases minoritarias).

## SVM Performance:


**Análisis:**

* **Precisión General:** Aproximadamente 66.5% (ligeramente mejor que Regresión Logística).
* **Clase 5:** Muy buen rendimiento (alta exhaustividad).
* **Clases 2, 3, 4:** Baja exhaustividad (similares problemas a la Regresión Logística).
* **Promedio Macro:** Similar a la Regresión Logística (dificultades con la clasificación uniforme de todas las clases).

## Resumen General:

* **Rendimiento General:** SVM y Regresión Logística superan a Naïve Bayes en precisión general.
* **Clase 5:** Los tres modelos tienen un rendimiento sólido en la clasificación de la clase 5.
* **Clases Minoritarias (2, 3, 4):** Los tres modelos muestran debilidades significativas en la identificación correcta de estas clases (baja exhaustividad).
* **Sesgo de Clase:** Los resultados sugieren un posible sesgo hacia la clase mayoritaria (clase 5).
* **Promedios:** Los promedios ponderados son más altos que los promedios macro, lo que refleja el impacto del buen rendimiento en la clase con mayor soporte.

## Implicaciones:

Se podría considerar explorar técnicas para mejorar el rendimiento en las clases minoritarias, como el sobremuestreo, el submuestreo o el uso de algoritmos más sensibles a las clases desequilibradas.
